In [83]:
#preprocessing
import numpy as np
import random

#plotting
import matplotlib.pyplot as plt

#connectivity measures
import nilearn.connectome as nic

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler

In [81]:
def analysis(y_test, y_pred, threshold=None):
    if threshold is not None:
        y_test_binary = (y_pred > threshold).astype(int)
        y_pred_binary = (all_ytest > threshold).astype(int)
    
    # Create the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    print("Confusion Matrix:")
    print(conf_matrix)

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    print('Precision', precision)
    print('Recall', recall)
    

In [2]:
%run loading_data.ipynb

The Python package you are importing, AtlasReader, is licensed under the
BSD-3 license; however, the atlases it uses are separately licensed under more
restrictive frameworks.
By using AtlasReader, you agree to abide by the license terms of the
individual atlases. Information on these terms can be found online at:
https://github.com/miykael/atlasreader/tree/master/atlasreader/data



In [3]:
sub_data, sub_ids = load_txt()
handedness = get_handedness(sub_ids)

Subject Data Loaded
restricted_data loaded


In [4]:
atlas = load_atlas()
atlas_coords = get_atlas_coords(atlas)

/tmp/ipykernel_362/1030149754.py:16: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  atlas = image.threshold_img(atlas, "99.5%")


atlas has shape ['x', 'y', 'z', 'region'] = (91, 109, 91, 100)


In [7]:
correlation_measure = nic.ConnectivityMeasure(kind='correlation')
correlation_matrix = correlation_measure.fit_transform(sub_data)

In [26]:
rois = [37, 40, 42]

In [32]:
dataX = correlation_matrix[:, :, rois]

In [69]:
X = []
for data in dataX:
    inner = []
    for i in range(3):
        row = data[:, 1]
        row = np.array(list(filter(lambda x: x != 1.000, row)))
        inner.extend(np.arctanh(row))
    X.append(inner)

In [70]:
X = np.array(X)

In [74]:
handedness['Class'] = handedness['Handedness']
handedness.loc[handedness['Class'] > 0] = 1
handedness.loc[handedness['Class'] < 0] = 0

In [90]:
for i in range(100):
    #train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, handedness['Handedness'])

    #oversampling X_train
    ros = RandomOverSampler(random_state=0)
    X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
    from collections import Counter
    print(sorted(Counter(y_resampled).items()))

    model = SVC(gamma='auto')
    model.fit(X_resampled, y_resampled)

    analysis(y_test, y_pred)

In [91]:
#oversampling X_train
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
from collections import Counter
print(sorted(Counter(y_resampled).items()))

[(0, 684), (1, 684)]


In [92]:
model = SVC(gamma='auto')
model.fit(X_resampled, y_resampled)

SVC(gamma='auto')

In [93]:
y_pred = model.predict(X_test)

In [94]:
analysis(y_test, y_pred)

Confusion Matrix:
[[  9  14]
 [ 88 140]]
Precision 0.9090909090909091
Recall 0.6140350877192983
